In [2]:
import pandas as pd
import matplotlib as plt

In [4]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql extension

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 6.0 MB 9.9 MB/s eta 0:00:01
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1207761 sha256=cf1975db901e7a95e31aec093656070eaebee76ed39a4abb1bad465a15b1a311
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.17
    Uninstalling SQLAlchemy-1.3.17:
      Successfully uninstalled SQLAlchemy-1.3.

In [5]:
%load_ext sql

In [7]:
%sql ibm_db_sa://username:password@host:50000/BLUDB

    QUERY

Find the total number of crimes recorded in the CRIME table

In [10]:
%%sql

SELECT COUNT (*) AS "TOTAL RECORDED CRIMES" FROM CHICAGO_CRIME_DATA

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


TOTAL RECORDED CRIMES
533


List community areas with per capita income less than 11000

In [11]:
%%sql

SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME < 11000

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
West Garfield Park
South Lawndale
Fuller Park
Riverdale


All case numbers for crimes involving minors

In [17]:
%%sql

SELECT CASE_NUMBER FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%MINOR%';

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


case_number
HL266884
HK238408


All kidnapping crimes involving a child

In [36]:
%%sql

SELECT * FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,latitude,longitude,location
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,1533,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


Crimes recorded at schools

In [42]:
%%sql

SELECT primary_type FROM CHICAGO_CRIME_DATA
WHERE location_description like '%SCH%'

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


primary_type
BATTERY
BATTERY
BATTERY
BATTERY
BATTERY
CRIMINAL DAMAGE
NARCOTICS
NARCOTICS
ASSAULT
CRIMINAL TRESPASS


Average safety score for all types of schools

In [113]:
%%sql

SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS "AVG_SAFETY_SCORE"
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School"
ORDER BY AVG_SAFETY_SCORE;

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


"Elementary, Middle, or High School",avg_safety_score
MS,48.000000
ES,49.520383
HS,49.623529


5 community areas with highest % of households below poverty line

In [80]:
%%sql

SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
Riverdale
Fuller Park
Englewood
North Lawndale
East Garfield Park


Most crime prone community area

In [11]:
%%sql

SELECT COMMUNITY_AREA_NUMBER, COUNT (*) AS "NUMBER_OF_CASES"
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY NUMBER_OF_CASES DESC 
LIMIT 6

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_number,number_of_cases
25,43
None,43
23,22
68,21
28,16
29,16


Using sub-query to find the name of the community area with highest hardship index

In [144]:
%%sql
SELECT COMMUNITY_AREA_NAME 
FROM CENSUS_DATA 
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
Riverdale


Using sub-query to determine the Community Area Name with most number of crimes

In [12]:
%%sql

SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER 
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT (COMMUNITY_AREA_NUMBER) DESC
    LIMIT 1)

 * ibm_db_sa://gwn73718:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
Austin
